In [1]:
import cv2
import json
import os
from keypoint_serialization import *
import numpy as np

In [2]:
sift = cv2.xfeatures2d.SIFT_create()

In [3]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

In [16]:
def get_kpd(file_name):
    with open(f"./prod/kpd_cache/{file_name[0:2]}/{file_name}", "r") as f:
        kpd = json.load(f)
    kpd = ([deserialize_kp(kp) for kp in kpd[0]], np.array(kpd[1]).astype(np.float32))
    return kpd

In [20]:
def get_uncropped_from_cropped(cropped):
    cropped_keypoints, cropped_descriptors = sift.detectAndCompute(cropped, None)
    closest_match = (None, None, None)
    for subdir in os.listdir("./prod/kpd_cache/"):
        for uncropped_file_name in os.listdir("./prod/kpd_cache/" + subdir):
            uncropped_keypoints, uncropped_descriptors = get_kpd(uncropped_file_name)
            matches = flann.knnMatch(cropped_descriptors, uncropped_descriptors, k=2)
            good = []
            for m, n in matches:
                if m.distance < 0.7 * n.distance:
                    good.append(m)
            uncropped_points = np.float32([uncropped_keypoints[m.trainIdx].pt for m in good])
            cropped_points = np.float32([cropped_keypoints[m.queryIdx].pt for m in good])
            M = cv2.findHomography(cropped_points, uncropped_points, cv2.RANSAC, 5.0)[0]
            if M is not None:
                uncropped = cv2.imread("./prod/sources/" + uncropped_file_name[0:-4])
                dst = cv2.warpPerspective(uncropped, M, (cropped.shape[1], cropped.shape[0]), flags=cv2.WARP_INVERSE_MAP)
                diff = np.average(dst.astype(float) - cropped)
                diff = abs(diff)
                #print(f"- diff: {diff}")
                if (not closest_match[0]) or closest_match[2] > diff:
                    closest_match = (uncropped_file_name, M, diff)
                    if diff == 0.0:
                        return closest_match
    return closest_match

In [21]:
cropped = cv2.imread("cropped.png")

In [22]:
get_uncropped_from_cropped(cropped)

('77ca53b430a31026e276957a1aae567a.jpg.json',
 array([[ 1.00011718e+00, -6.65707399e-05,  2.02000520e+02],
        [ 4.10740713e-05,  1.00000753e+00,  5.99899247e+01],
        [ 1.12251069e-07, -6.90603442e-08,  1.00000000e+00]]),
 0.03677416643279468)